# Gatsiva Symbol Price Example

This notebook shows how retrieve cryptocurrency prices utilizing Gatsiva [Symbol Price Data](https://gatsiva.com/docs/#get-symbol-price-data) transaction.


API access is **currently limited to beta testers and collaborators**. For more information on how to utilize the Gatsiva API or to request access and an API key, please visit the [Gatsiva Website](https://gatsiva.com). For more information on the Gatsiva API, please visit the [Gatsiva API Documentation](https://gatsiva.com/docs).

## Enter Input Information Here

In [ ]:
# Enter the symbol and condition to use
symbol = 'ETH:USD:daily'

# Enter your API key for Gatsiva
apikey = "<your_api_key>"

#### Python code to get the results - see results further below


In [ ]:
# Install Prerequesities if needed
#
# Run this (uncomment below) to install / upgrade Plotly
#!pip install plotly --upgrade


In [ ]:
# Import utility libraries we will need
import requests
import json
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib.lines import Line2D

# Import datetime for some time processing
from datetime import datetime

# Ignore warnings
warnings.filterwarnings('ignore')

# Allow matplotlib to render inline (needed for pandas plotting)
%matplotlib inline


#### Define Price Data API Call

In [ ]:
# Define the function for calling the historical data which will return a pandas data frame
def call_price_data(symbol, apikey):
    bearer_token = 'Bearer ' + apikey
    my_headers = {'Accept': 'application/json', 'Authorization': bearer_token}
    analytic = requests.get('https://api.gatsiva.com/api/v1/symbols/' + symbol,headers=my_headers)
    results = analytic.json()['results']
    
    # The returning data structure looks like this
    #
    #    {
    #      "1505462664": {
    #        "open": 4203.02,
    #        "high": 4302.11,
    #        "low": 3990.02,
    #        "close": 4102.88,
    #        "volume": 123012312
    #      }
    #    },
    #    {
    #      "1505549064": {
    #        "open": 4203.02,
    #        "high": 4302.11,
    #        "low": 3990.02,
    #        "close": 4102.88,
    #        "volume": 123012312
    #      }
    #    },
    
    # Because the timestamp marks the beginning of the period, we need to modify the timestamp appropriately
    price_type = symbol.split(':')[2]

    ts_offset = 0

    if price_type == 'daily':
        ts_offset = 86400 # Adds 1 day
    elif price_type == 'hourly':
        ts_offset = 3600 # Adds 1 hour

    # We need to convert the string keys into timestamps
    keys = []
    for n in results.keys():
        # We add the ts_offset to make sure that the label indicates the time at which this was true, which is the end of the period
        keys.append(pd.Timestamp(datetime.utcfromtimestamp(int(n) + ts_offset)))

    # Create the dataframe from the dictioanry results
    newdf = pd.DataFrame.from_dict(results, orient='index')
    
    # Utilize our converted timestamps as the index
    newdf.index = keys
    
    return newdf

In [ ]:
# API Call - Get the price data
price_data = call_price_data(symbol, apikey)

# Results

In [ ]:
price_data.tail()